In [90]:
## Preamble: Package Loading
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import itertools as iter
import math 
import sys
import os
import json

<h2> Panel Selection and Control: Monte Carlo Data Generating Process  </h2> 

<h3> 1.0 DGP Description </h3>

Here I decribe the manner in which the data used in the monte carlo simulation is generated. 

<h3> 1.1 Error, Instrument, and Exogenous Variable Generation </h3>

Let; $n_{tp} \equiv T$ be the total number of time periods, $n_{end} \equiv p_1$ be the number of endogneous regressors included in the primary regression, $n_{exo} \equiv p_2$ be the number of exogenous regressors included in the primary regression, and $n_{tinst} \equiv w$, $ n_{cinst} \equiv w_j$ be the total number of available instruments and the number of instruments relevant to each crossection respectively. Now let,

$$
\begin{align*} 
\rho_{er} &= \begin{bmatrix} \rho_{er,1} & \rho_{er,2} & \cdots & \rho_{er,n_{end}} \end{bmatrix} \\
\rho_{inst} &= \begin{bmatrix} \rho_{inst,1} & \rho_{inst,2} & \cdots & \rho_{inst,n_{inst}-1} \end{bmatrix}\\
\rho_{ex} &= \begin{bmatrix} \rho_{ex,1} & \rho_{ex,2} & \cdots & \rho_{ex,n_{ex}-1} \end{bmatrix}  
\end{align*}
$$

So that I can define the covariance matrices for *each* cross section as follows 

$$
\begin{align*}
cv_{er} &= \begin{bmatrix} 
1 & \rho_{er,1} & \rho_{er,2} & \cdots & \rho_{er,n_{end}} \\
\rho_{er,1} & 1  & \rho_{er,1} &\cdots & \rho_{er,n_{end}-1} \\
\rho_{er,2} & \rho_{er,1} & 1 & \cdots & \rho_{er,n_{end}-2} \\
\vdots & &&\ddots&  \\
 \rho_{er,n_{end}} & \rho_{er,n_{end}-1} & \rho_{er,n_{end}-2} & \cdots &  1 
\end{bmatrix}
%
\hspace{1cm} 
%
%
cv_{ex} &= \begin{bmatrix} 
1 & \rho_{ex,1} & \rho_{ex,2} & \cdots & \rho_{ex,n_{ex}-1} \\
\rho_{ex,1} & 1  & \rho_{ex,1} &\cdots & \rho_{ex,n_{ex}-2} \\
\rho_{ex,2} & \rho_{ex,1} & 1 & \cdots & \rho_{ex,n_{ex}-3} \\
\vdots & &&\ddots&  \\
 \rho_{ex,n_{ex}-1} & \rho_{ex,n_{ex}-2} & \rho_{ex,n_{ex}-2} & \cdots &  1 
\end{bmatrix}
\end{align*} 
$$

Since all crossections will use some subset of the vector of instruments the following is the full covariance matrix for all instruments.

$$
CV_{inst} = \begin{bmatrix} 
1 & \rho_{inst,1} & \rho_{inst,2} & \cdots & \rho_{inst,n_{tinst}-1} \\
\rho_{inst,1} & 1  & \rho_{inst,1} &\cdots & \rho_{inst,n_{tinst}-2} \\
\rho_{inst,2} & \rho_{tinst,1} & 1 & \cdots & \rho_{inst,n_{tinst}-3} \\
\vdots & &&\ddots&  \\
 \rho_{inst,n_{tinst}-1} & \rho_{inst,n_{tinst}-2} & \rho_{inst,n_{tinst}-3} & \cdots &  1 
\end{bmatrix}
%
$$

As a result we can construct the covariance matrices for all cross sections,

$$
\begin{align*}
CV_{er} &= 
\begin{bmatrix}
cv_{er} & \mathbf{0}_{(n_{end}+1 \times n_{end}+1)} & \cdots & \mathbf{0}_{(n_{end}+1 \times n_{end}+1)}  \\
\mathbf{0}_{(n_{end}+1 \times n_{end}+1)} & cv_{er} & \cdots & \mathbf{0}_{(n_{end}+1 \times n_{end}+1)}  \\
\vdots & \vdots & \ddots & \vdots \\
\mathbf{0}_{(n_{end}+1 \times n_{end}+1)} & \mathbf{0}_{(n_{end}+1 \times n_{end}+1)} & \cdots & cv_{er}
\end{bmatrix} 
%
\hspace{1cm}
%
CV_{ex}  = 
\begin{bmatrix}
cv_{ex} & \mathbf{0}_{(n_{ex} \times n_{ex})} & \cdots & \mathbf{0}_{(n_{ex} \times n_{ex})}  \\
\mathbf{0}_{(n_{ex} \times n_{ex})} & cv_{ex} & \cdots & \mathbf{0}_{(n_{ex} \times n_{ex})}  \\
\vdots & \vdots & \ddots & \vdots \\
\mathbf{0}_{(n_{ex} \times n_{ex})} & \mathbf{0}_{(n_{ex} \times n_{ex})} & \cdots & cv_{ex}
\end{bmatrix} 
\end{align*}
$$


=Now I generate, error terms, instruments, and exogneous variables from mulitvariate normal distribution. First let

$$ 
\begin{align*} 
Z_{2jt} &= \begin{bmatrix} Z_{2jt,1} & Z_{2jt,2} & \cdots & Z_{2jt,n_{ex}} \end{bmatrix}' \\[10pt]  
W_t &= \begin{bmatrix} W_{t,1} & W_{t,2} & \cdots & W_{t,n_{inst}} \end{bmatrix}' \\[10pt]
\tilde{V}_{jt} &= \begin{bmatrix} V_{jt,1} & V_{jt,2}& \cdots & V_{jt,n_{end}} & \varepsilon_{j} \end{bmatrix}' 
\end{align*} 
$$

Then consider, $ W_{t} \sim N(\mathbf{0}_{n_{inst} \times 1}, CV_{inst})$

$$
\begin{bmatrix} Z_{21t}' & Z_{22t}' & \cdots & Z_{2n_{cs}t}' \end{bmatrix}' \sim N(\mathbf{0}_{n_{cs} \cdot n_{exo} \times 1}, CV_{ex})
\hspace{1cm} \text{ and } \hspace{1cm} 
\begin{bmatrix} \tilde{V}_{1t}' & \tilde{V}_{2t}' & \cdots & \tilde{V}_{n_{cs},t}' \end{bmatrix}' \sim N(\mathbf{0}_{n_{cs} \cdot (n_{end} +1) \times 1}, CV_{er})
$$

<h3> 1.2 Endogenous Variable Generation </h3>  

Now we wish to generate the endogenous variables from the above but in accordance with the material presented in the proposal we consider two different sturctures. 

1. Non Panel: Parameter Vectors are not shared across cross sections meaning that secondary equations for each cross section are seperate.


2. Panel: Parameter vectors are shared across cross sections.

<h4> 1.2.1 Endogenous Variable Generation: Non Panel </h4>

Since the secondary equations are not of a panel type, I will

* For each $j\in \{1,2,\cdots , n_{cs}\}$ and $d \in \{1,2,\cdots,n_{end} \}$ I will draw the coefficienct vector $\alpha_{1jd}$ from the cartesian product of $[1,1]$ with itself $n_{exo}$ times. 


* For each $j\in \{1,2,\cdots , n_{cs}\}$ I will draw a sequence of integers $C_{j}$ from the $\mathcal{C}^{n_{tinst}}_{n_{cinst}}$ ways that that you can choose $n_{cinst}$ instruments from $n_{tinst}$ total instrument to be included in every regression of $Z_{1j}$ on $Z_{2j}$ and $W$ i.e. these numbers define which columns of $W$ are included in $W_j$.


* For each $j\in \{1,2,\cdots , n_{cs}\}$ and $d \in \{1,2,\cdots,n_{end} \}$ I will draw the coefficienct vector $\alpha_{2jd}$ from the cartesian product of $[1,1]$ with itself $n_{cinst}$ times. 


* Then for each $j\in \{1,2,\cdots , n_{cs}\}$ and $d \in \{1,2,\cdots,n_{end} \}$ I generate endogenous regressors $Z_{1jd}$ as follows

$$ Z_{1jd} =  \alpha_{0jd} + Z_{2jt}' \alpha_{1jd} + W_{jt}' \alpha_{2jd} + V_{jt,d} \hspace{1cm} \text{ where } \hspace{1cm} \alpha_{0jd} = 1/2+j/2 $$ 



<h4> 1.2.2 Endogenous Variable Generation: Panel </h4>

If the secondary equations are panel type, I will

* For each $d \in \{1,2,\cdots,n_{end} \}$ I will draw the coefficienct vector $\alpha_{1d}$ from the cartesian product of $[1,1]$ with itself $n_{exo}$ times. 


* For each $j\in \{1,2,\cdots , n_{cs}\}$ I will draw a sequence of integers $C_{j}$ from the $\mathcal{C}^{n_{tinst}}_{n_{cinst}}$ ways that that you can choose $n_{cinst}$ instruments from $n_{tinst}$ total instrument to be included in every regression of $Z_{1j}$ on $Z_{2j}$ and $W$ i.e. these numbers define which columns of $W$ are included in $W_j$.


* For each $d \in \{1,2,\cdots,n_{end} \}$ I will draw the coefficienct vector $\alpha_{2d}$ from the cartesian product of $[1,1]$ with itself $n_{tinst}$ times. 


* Then for each $j\in \{1,2,\cdots , n_{cs}\}$ and $d \in \{1,2,\cdots,n_{end} \}$ I generate endogenous regressors $Z_{1jd}$ as follows

$$ Z_{1jd} =  \alpha_{0jd} + Z_{2jt}' \alpha_{1d} + W_{jt}' \alpha_{2d} + V_{jt,d} \hspace{1cm} \text{ where } \hspace{1cm} \alpha_{0jd} = 1/2+j/2 $$ 


<h3> Primary Regressand Generation </h3>

Having generated all primary regressors I will generate the regresand for the primary equation to do this I will,

* I will draw the coefficienct vector $\beta_1$ from the cartesian product of $[1,1]$ with itself $n_{exo} +n_{end}$ times. Then set


$$ Y_{jt} = [\; Z_{1jt}' \;\; Z_{2jt}' \;] \beta_1 + e_j + \varepsilon_{jt} \;\;\;\; \text{ where } \;\;\;\;  e_{j} = 1+j/2  $$



<h3> 2.0 Block Diagonal Matrix Function </h3>

In order to facilitate construction of the block diagonal matrices $V_{er},V_{ex}$, and $V_{inst}$ discussed above, I define a following function

In [66]:
def blkdiag(mat,nb):
    """
INPUTS
mat     Square Matrix which will form the block in a block diagonal matrix
nb      Number of diagonal block in output matrix 

OUTPUT
v       Block diagonal matrix of dimension ( nb*mat.shape[0] x nb*mat.shape[0] )
    """
    # Initializing the varcov matrix for all crosssections
    v = np.hstack((mat,np.zeros((mat.shape[0],(nb-1)*(mat.shape[1])))))
    # Registry matrix used in following loops
    vreg = np.eye(nb-1)
    for j in np.arange(nb-1):
        # Initializing current block of rows 
        pv = np.zeros((mat.shape[0],mat.shape[0]))
        # Horiz Stacking either zeros or var_err depending on ve_reg[j,i]
        for i in np.arange(nb-1):
            if vreg[j,i] == 1: # Stack var_err onto pv_err
                pv = np.hstack((pv,mat))
            if vreg[j,i] == 0: # Stack zeros onto pv_err
                pv = np.hstack((pv,np.zeros((mat.shape[0],mat.shape[0]))))
        # Vertically stacking block rows on top of one another
        v = np.vstack((v,pv))
    return v

<h3> 2.1 Block Diagonal Matrix Function Demonstration </h3>

In [67]:
blkdiag(np.ones((3,3)),4)

array([[ 1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.]])

<h3> 3.0 DGP Generation Code </h3>

What follows is the function used to generate all monte carol simulation data sets. 

In [127]:
def psc_dgp(inpt_d): 
    '''
PURPOSE: 
Generate Draws from the dgp detailed in penner(2018) 'Panel Selection and Control'

INPUTS: 
inpt_d      Dictionary with the following keyword items
 r_seed      Integer random number generator seed
 nds         Number of generated data sets (int)
 ntp         Number of time periods (int)
 ncs         Number of cross sections (int)
 n_end       Number of endogenous variables in primary regression (int)
 n_exo       Number of exogenous variables in primary regression (int)
 t_inst      Number of total instruments available to all cross sections (int)
 c_inst      Number of valid instrument per cross section (int < t_inst )
 frc         Indicator for forcing control function to be other than correlation based
 sec_pan     Indicator for whether secondary equation is panel (=1) or not (=0)
 ex_vpro     List of exogenous regr covariances where cov(Z21,i , Z21,(i+j)) = ex_vpro[j-1] 
 inst_vpro   List of instrument covariances where cov(Wi , W(i+j)) = inst_vpro[j-1] 
 err_vpro    List of error covariances where cov(V1,i , V1,(i+j)) = err_vpro[j-1] 
 
OUPUTS: 
data_sets                      List with the following elements
 data_sets[0][0]                  dictionary inpt_d  returned
 data_sets[0][1]['Derr_nms']      list of column names for each error term df
 data_sets[0][1]['Dins_nms']      list of column names for each instrument df
 data_sets[0][1]['Dlng_nms']      list of column names for each long df
 data_sets[0][1]['coeff']         list of lists of coefficient matrices used in secondary eqn 
 data_sets[0][1]['pcoeff']        list of n_exo+n_end coefficents used to generate primary regressand
 data_sets[0][1]['var_inst']      list of lists when coverted to array is instrument VCOV matrix
 data_sets[0][1]['V_ex']          list of lists when coverted to array is exog regs VCOV matrix
 data_sets[0][1]['V_err']         list of lists when coverted to array is error term VCOV matrix
 data_sets[i][0]['err_df']        ith error term data array in list form
 data_sets[i][0]['prim_df']       ith primary regression variables data array in list form
 data_sets[i][0]['inst_df]        ith instruments data array in list form    
    '''

    # Extracting all input variables from inpt_d dictionary
    r_seed = inpt_d['r_seed']
    nds = inpt_d['nds']
    ntp = inpt_d['ntp']
    ncs = inpt_d['ncs']
    n_end = inpt_d['n_end']
    n_exo = inpt_d['n_exo']
    t_inst = inpt_d['t_inst']
    c_inst = inpt_d['c_inst']
    frc = inpt_d['frc']
    sec_pan = inpt_d['sec_pan']
    ex_vpro = inpt_d['ex_vpro']
    inst_vpro = inpt_d['inst_vpro']
    err_vpro = inpt_d['err_vpro']
    np.random.seed([r_seed])

    ## Coefficients on instruments in secondary equation
    if sec_pan == 0:
        # Not panel so coefficients have seperate coeff vector of length c_inst
        icoeffs_reg = list(iter.product([-1,1],repeat = c_inst))
        # All permutation of ncs pairs of n_end coeff vectors on t_inst instruments  
        picfs = list(iter.permutations(range(0,len(icoeffs_reg)),n_end))
        # Rand choosing 1 (if panel) or ncs (if !panel) coeff for ex regress in secondary reg.
        icfs = [ picfs[i] for i in np.random.randint(len(picfs),size = ncs)]
    else: 
        # Is panel so coefficients have common vector of length t_inst
        icoeffs_reg = list(iter.product([-1,1],repeat = t_inst))
        # All permutation of ncs pairs of n_end coeff vectors on t_inst instruments 
        if len(icoeffs_reg) < 99:
            picfs = list(iter.permutations(range(0,len(icoeffs_reg)),n_end))
            # Rand choosing 1 (if panel) coeff for ex regress in secondary reg.
            icfs = [ picfs[i] for i in np.random.randint(len(picfs),size =1)]*ncs
        else:
            icfs = [tuple(np.random.randint(len(icoeffs_reg),size =n_end))]*ncs

    ## Assignment of the relvant instruments to each cross section.
    # Registry of instrument assignments
    insts_reg = list(iter.combinations(np.arange(1,t_inst+1),c_inst))
    # List of which instruments (col #'s) are relevant for each crossection 
    icr = [insts_reg[np.random.randint(len(insts_reg))] for i in range(ncs)]

    ## Coefficients on exogenous variables in secondary equations       
    # Collection of all Coefficients on Exogenous Variables in secondary eqns
    excoeffs_reg = list(iter.product([-1,1],repeat = n_exo))
    # All permutation of combinations of ncs coeff vectors on  for ex regressors 
    pxcfs = list(iter.permutations(range(0,len(excoeffs_reg)),n_end))
    # Rand choosing 1 (if panel) or ncs (if !panel) coeff for ex regress in secondary reg.
    if sec_pan == 0:
        # Not panel so rand choosing ncs collections of n_exo reg numbers from pxcfs
        xcfs = [ pxcfs[i] for i in np.random.randint(len(pxcfs),size = ncs)]
    else:
        # Is panel so need only 1 collection of n_exo reg numbers from pxcfs & duplicating
        xcfs = [ pxcfs[i] for i in np.random.randint(len(pxcfs),size = 1)]*ncs

    ## Generation of the coefficient matrix for secondary regression.             
    # Initializing Coefficient Matrix            
    coeff = np.zeros((n_end,ncs,t_inst+n_exo))
    for j in range(n_end):
        for i in range(ncs):
            for k in range(n_exo):
                coeff[j,i,k] = excoeffs_reg[xcfs[i][j]][k]
            if sec_pan == 0:
                for l in range(c_inst):
                    k = icr[i][l]
                    coeff[j,i,k+n_exo-1] = icoeffs_reg[icfs[i][j]][l]   
            else: 
                for l in icr[i]:
                    coeff[j,i,l+n_exo-1] = icoeffs_reg[icfs[i][j]][l-1] 
                    
    ## Generation primary regression coefficient vector                
    # Common Primary Coeff Vector
    pcoeff = np.array([1,-1]*int(np.ceil((2*(n_end + n_exo)+1)/2)))[:n_end+n_exo].reshape(n_end+n_exo,1)
    # Fixed Effect for each crossection
    fe = [ 1+x/2 for x in np.arange(0,ncs)]

    ## Joint Distribution of Exogenous regressors
    # Vector of Means (=0)
    mu_ex = np.zeros(n_exo)
    # Diagonal matrix of variaces (=1)
    var_ex = np.eye(n_exo)
    # Variance Covariance Matrix Generation or EACH crossection
    for i in np.arange(len(ex_vpro)):
        var_ex = (var_ex + ex_vpro[i]*np.eye(n_exo,k=i+1)
                             + ex_vpro[i]*np.eye(n_exo,k=-(i+1))) 

    ## Joint Distribution of Instruments for all cross sections
    # Vector of Means (=0)
    mu_inst = np.zeros(t_inst)
    # Diagonal Matrix of Variaces (=1)
    var_inst = np.eye(t_inst)
    # Variance Covariance Matrix Generation
    for i in np.arange(len(inst_vpro)):
        var_inst = (var_inst + inst_vpro[i]*np.eye(t_inst,k=i+1) 
                             + inst_vpro[i]*np.eye(t_inst,k=-(i+1)))

    ## Joint Distribution of Error Terms for EACH crossection
    # Vector of means
    mu_err = np.zeros(n_end+1)
    # Diagonal Matrix of Variances 
    var_err = np.eye(n_end+1)
    # Variance Covariance Matrix Generation
    if frc == 0 : 
        # Var Cov matrix for correlated errors ==> additive linear control functions 
        for i in np.arange(len(err_vpro)):
            var_err = (var_err + err_vpro[i]*np.eye(n_end+1,k=i+1) 
                                 + err_vpro[i]*np.eye(n_end+1,k=-(i+1)))
    else: 
        # Error not explicitly correlated ==> have to force control functions.
        var_err = np.eye(n_end+1)

    # Error term mean vector for ALL Crossections
    Mu_err = np.tile(mu_err,ncs)        
    # Error term variance covariance matrix for ALL Crossections
    V_err = blkdiag(var_err,ncs)

    # Exogenous regressor mean vector for ALL Crossections
    Mu_ex = np.tile(mu_ex,ncs) 
    # Exogenous regressor variance covariance matrix for ALL Crossections
    V_ex = blkdiag(var_ex,ncs)

    ## Variable Name Generation
    # exogenous variable name generation 
    #          [Z21,1 , Z21,2 ......, Z22,1 , Z22,2 , ..... ]        
    ex_nms = [''.join(['Z2',str(i),',',str(j)]) 
              for i in list(range(1,ncs+1)) 
              for j in list(range(1,n_exo+1))]
    # instruments names generation 
    #          [W1 , W2 , .... ]
    inst_nms = [''.join(['W',str(i)]) for i in list(range(1,t_inst+1))]
    # Error terms names generation
    #          [V1,1 , V1,2 , .... ,e1 , V2,1 , V2,2 , ...... e2 , ......]
    err_nm1 = ['e' if val == n_end+1 else 'V' for val in  list(range(1,n_end+2))*ncs]
    err_nm2 = [ str(i) for y in range(1,ncs+1) for i in iter.repeat(y,n_end+1)]
    err_nm3 = ['' if val == n_end+1 else ''.join([',',str(val)]) 
               for val in list(range(1,n_end+2))*ncs]
    err_nm  = [''.join([err_nm1[i],err_nm2[i],err_nm3[i]]) for i in range(len(err_nm1))]

    # Initializing the data sets list
    data_sets = []

    for k in range(nds):
        ## Variable Generation
        time = np.arange(1,ntp+1).reshape(ntp,1)
        # Exogenous Regressor Generation
        Ex = np.random.multivariate_normal(Mu_ex,V_ex,ntp)
        Ex = np.hstack((time,Ex))
        # Instruments Generation
        Inst = np.random.multivariate_normal(mu_inst,var_inst,ntp)
        Inst = np.hstack((time,Inst))
        # Error Terms Generation 
        Err = np.random.multivariate_normal(Mu_err,V_err,ntp)
        Err = np.hstack((time,Err))
        ## Data Frame Generation 
        Ex_df = pd.DataFrame(Ex,columns = ['t'] + ex_nms)
        Inst_df = pd.DataFrame(Inst, columns = ['t'] + inst_nms)
        Err_df = pd.DataFrame(Err,columns = ['t'] + err_nm)

        ## Generating Endogenous (primary) regressors
        for j in range(n_end):
            for i in range(ncs):
                # Regular expression for the relevant exogenous regressors
                ex_pat = ''.join(['^Z2',str(i+1),','])
                # Regular expression for the relevant error term. 
                err_pat = ''.join(['V',str(i+1),',',str(j+1)])
                # Extracting exog regresors converting to numpy array
                pe1 = pd.concat([Ex_df.filter(regex = ex_pat),Inst_df.iloc[:,1:]], axis = 1).values
                # Extracting error variable and converting to numpy array
                pe2 = Err_df.filter(regex = err_pat).values
                # Calculating the endogenous primary regressor
                pe = pe1.dot(coeff[j,i,:]).reshape(pe1.shape[0],1) + pe2
                # Constructing the appropriate name for the endo regressor
                end_nm = ''.join(['Z1',str(i+1),',',str(j+1)])
                if j == 0 and i == 0:
                    # Initializing the endog df with first calculated regressor
                    End_df = pd.DataFrame(pe,columns = [end_nm])
                else:
                    # Adding calculated endog regressor onto df
                    End_df[end_nm] = pe

        ## Generation of primary regressand
        for i in range(ncs):
            # Regular expression for the relevant endogenous regressors
            en_pat = ''.join(['^Z1',str(i+1),','])
            # Regular expression for the relevant exogenous regressors
            ex_pat = ''.join(['^Z2',str(i+1),','])
            # Name of apporpriate primary error term
            er_nm = ''.join(['e',str(i+1)])
            # Extracting appropriate regressor for primary equation
            pr3 = pd.concat([End_df.filter(regex = en_pat),
                             Ex_df.filter(regex = ex_pat)], axis = 1).values
            # Extracting appropriate error term
            pr4 = Err_df[er_nm].values.reshape(Err_df.shape[0],1)
            # Generating primary regressand
            if frc == 0:
                # if no forcing no need to include control functions explicity
                pr = fe[i] + pr3.dot(pcoeff)+pr4
            else:
                # need to include control functions explicitly
                # (inprogress)
                pass
            # Constructing the appropriate name for the endo regressor
            pr_nm = ''.join(['Y',str(i+1)])
            if i == 0:
                # Initializing the regressand df 
                Pr_df = pd.DataFrame(pr,columns = [pr_nm])   
            else:
                # Adding generated regressand to df
                Pr_df[pr_nm] = pr

        ## Converting Data To Long Panel Type
        for i in range(ncs):
            # Initializing temporary df
            pL = None
            # Columns Names for endogeneous regressors
            Z1_nm = [ ''.join(['Z1',',',str(j)]) for j in range(1,n_end+1)]
            # Columns Names for exogenous regressors
            Z2_nm = [ ''.join(['Z2',',',str(j)]) for j in range(1,n_exo+1)]
            # Adding regressand columns to pL
            pL = pd.DataFrame(Pr_df[''.join(['Y',str(i+1)])].values,columns = ['Y'])
            # Adding endog regressors to pL
            pL = pd.concat([pL,pd.DataFrame(
                             End_df.filter(regex = ''.join(['^Z1',str(i+1),','])).values
                             ,columns = Z1_nm)],axis = 1)
            # Adding exog regressors to pL
            pL = pd.concat([pL,pd.DataFrame(
                             Ex_df.filter(regex = ''.join(['^Z2',str(i+1),','])).values
                             ,columns = Z2_nm)],axis = 1)
            # Adding the crossection variable
            pL['crs'] = i+1
            # Adding the time component variable
            pL['t'] = pd.DataFrame(np.arange(1,ntp+1).reshape(ntp,1))
            if i == 0 :
                # Initializing Data_long
                Data_long = pL
            else:
                # Adding pL to the bottom of Data_long
                Data_long = pd.concat([Data_long,pL], axis = 0)

        # Sorting Data_long by column name      
        Data_long = Data_long[list(Data_long.columns)[-2:] + list(Data_long.columns)[:-2]]

        # Extract names in Df_s only once
        if k == 0:
            # Names for export data sets
            Dlng_nms = list(Data_long.columns)
            Dins_nms = list(Inst_df.columns)
            Derr_nms = list(Err_df.columns)
            nms_cfs = {}
            nms_cfs['Dlng_nms'] = Dlng_nms
            nms_cfs['Dins_nms'] = Dins_nms
            nms_cfs['Derr_nms'] = Derr_nms
            nms_cfs['coeff'] = coeff.tolist()
            nms_cfs['pcoeff'] = pcoeff.T[0].tolist()
            nms_cfs['var_inst'] = var_inst.tolist()
            nms_cfs['V_ex'] = V_ex.tolist()
            nms_cfs['V_err'] = V_err.tolist() 
            data_sets.append([inpt_d , nms_cfs])

        # Adding constructed data sets to data_sets list
        c = {}
        c['err_df'] = np.array(Err_df).tolist()
        c['prim_df'] = np.array(Data_long).tolist()
        c['inst_df'] = np.array(Inst_df).tolist()
        data_sets.append([c])
        
    return data_sets

In [110]:
?psc_dgp

<h3> DGP Inputs </h3>

In [131]:
%%time
# Construction of the input dictionary (see: ?psc_dgp for details)
inpt_d = {'r_seed':199,'nds':1000, 'ntp':10 , 'ncs': 4 , 'n_end': 2,
             'n_exo': 2 , 't_inst': 10, 'c_inst': 4,
             'frc':0, 'sec_pan': 0, 'ex_vpro': [0.5],
             'inst_vpro': [0.5 , 0.25], 'err_vpro':[0.8 , 0.36 ]}
# Function call
psc_data = psc_dgp(inpt_d)

CPU times: user 1min 9s, sys: 1.13 s, total: 1min 10s
Wall time: 35.8 s


<h3> DGP JSON encoding and saving </h3>

In [133]:
wkng_folder = os.getcwd()
data_folder = ''.join([current_folder,'/','data'])
output_filename = 'pscdata_5_23_2.json'
output_file_full = ''.join([data_folder,'/',output_filename])

with open(output_file_full, 'w') as f_obj:
    json.dump(psc_data, f_obj)

<h3> Data set meta data dictionary </h3>

In [134]:
psc_data[0][0]

{'c_inst': 4,
 'err_vpro': [0.8, 0.36],
 'ex_vpro': [0.5],
 'frc': 0,
 'inst_vpro': [0.5, 0.25],
 'n_end': 2,
 'n_exo': 2,
 'ncs': 4,
 'nds': 1000,
 'ntp': 10,
 'r_seed': 199,
 'sec_pan': 0,
 't_inst': 10}

<h3> Coefficient vector for regression of $Z_{1j1}$ on $[Z_{2j} , W ]$ </h3>

In [77]:
c1 = [''.join(['$\\alpha_{1j1,',str(i),'}$']) for i in range(1,n_exo+1)]
for i in range(1,t_inst+1):
    c1.append(''.join(['$\\alpha_{2j1,',str(i),'}$']))

c11 = [''.join(['j=',str(i)]) for i in range(1,ncs+1)]
    
pd.DataFrame(np.array(psc_data[0][1]['coeff'][0]),columns = c1, index = c11)

,"$\alpha_{1j1,1}$","$\alpha_{1j1,2}$","$\alpha_{2j1,1}$","$\alpha_{2j1,2}$","$\alpha_{2j1,3}$","$\alpha_{2j1,4}$","$\alpha_{2j1,5}$","$\alpha_{2j1,6}$","$\alpha_{2j1,7}$","$\alpha_{2j1,8}$","$\alpha_{2j1,9}$","$\alpha_{2j1,10}$"
j=1,-1.0,1.0,0.0,1.0,1.0,0.0,-1.0,0.0,0.0,1.0,0.0,0.0
j=2,1.0,-1.0,0.0,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,-1.0
j=3,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,-1.0,1.0
j=4,1.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,-1.0,1.0,1.0,0.0


<h3> Coefficient vector for regression of $Z_{2j2}$ on $[Z_{2j} , W ]$ </h3>

In [78]:
c2 = [''.join(['$\\alpha_{1j2,',str(i),'}$']) for i in range(1,n_exo+1)]
for i in range(1,t_inst+1):
    c2.append(''.join(['$\\alpha_{2j2,',str(i),'}$']))

pd.DataFrame(np.array(psc_data[0][1]['coeff'][1]),columns = c2,index = c11)

,"$\alpha_{1j2,1}$","$\alpha_{1j2,2}$","$\alpha_{2j2,1}$","$\alpha_{2j2,2}$","$\alpha_{2j2,3}$","$\alpha_{2j2,4}$","$\alpha_{2j2,5}$","$\alpha_{2j2,6}$","$\alpha_{2j2,7}$","$\alpha_{2j2,8}$","$\alpha_{2j2,9}$","$\alpha_{2j2,10}$"
j=1,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
j=2,1.0,1.0,0.0,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,1.0
j=3,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0
j=4,1.0,-1.0,0.0,0.0,1.0,0.0,0.0,0.0,-1.0,-1.0,1.0,0.0


<h3> Error term variance covariance matrix </h3>

In [79]:
pd.DataFrame(np.array(psc_data[0][1]['V_err']))

,0,1,2,3,4,5,6,7,8,9,10,11
0,1.00,0.8,0.36,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00
1,0.80,1.0,0.80,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00
2,0.36,0.8,1.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00
3,0.00,0.0,0.00,1.00,0.8,0.36,0.00,0.0,0.00,0.00,0.0,0.00
4,0.00,0.0,0.00,0.80,1.0,0.80,0.00,0.0,0.00,0.00,0.0,0.00
5,0.00,0.0,0.00,0.36,0.8,1.00,0.00,0.0,0.00,0.00,0.0,0.00
6,0.00,0.0,0.00,0.00,0.0,0.00,1.00,0.8,0.36,0.00,0.0,0.00
7,0.00,0.0,0.00,0.00,0.0,0.00,0.80,1.0,0.80,0.00,0.0,0.00
8,0.00,0.0,0.00,0.00,0.0,0.00,0.36,0.8,1.00,0.00,0.0,0.00
9,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,1.00,0.8,0.36


<h3> Instruments variance covariance matrix </h3>

In [80]:
pd.DataFrame(np.array(psc_data[0][1]['var_inst']))

,0,1,2,3,4,5,6,7,8,9
0,1.00,0.50,0.25,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.50,1.00,0.50,0.25,0.00,0.00,0.00,0.00,0.00,0.00
2,0.25,0.50,1.00,0.50,0.25,0.00,0.00,0.00,0.00,0.00
3,0.00,0.25,0.50,1.00,0.50,0.25,0.00,0.00,0.00,0.00
4,0.00,0.00,0.25,0.50,1.00,0.50,0.25,0.00,0.00,0.00
5,0.00,0.00,0.00,0.25,0.50,1.00,0.50,0.25,0.00,0.00
6,0.00,0.00,0.00,0.00,0.25,0.50,1.00,0.50,0.25,0.00
7,0.00,0.00,0.00,0.00,0.00,0.25,0.50,1.00,0.50,0.25
8,0.00,0.00,0.00,0.00,0.00,0.00,0.25,0.50,1.00,0.50
9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.25,0.50,1.00


<h3> Exogenous regressors variance covariance </h3>

In [81]:
pd.DataFrame(np.array(psc_data[0][1]['V_ex']))

,0,1,2,3,4,5,6,7
0,1.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0
1,0.5,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.5,0.0,0.0,0.0,0.0
3,0.0,0.0,0.5,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.5,0.0,0.0
5,0.0,0.0,0.0,0.0,0.5,1.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.5
7,0.0,0.0,0.0,0.0,0.0,0.0,0.5,1.0


<h3> Long panel of primary regression variables </h3>

In [82]:
pd.DataFrame(np.array(psc_data[1][0]['prim_df']), columns = psc_data[0][1]['Dlng_nms'])

,crs,t,Y,"Z1,1","Z1,2","Z2,1","Z2,2"
0,1.0,1.0,4.731123,-0.158987,-5.246847,-1.106620,0.374755
1,1.0,2.0,-0.917255,-5.983521,-2.746257,0.517263,-0.470883
2,1.0,3.0,1.816183,0.061128,-1.156360,-0.010323,0.924917
3,1.0,4.0,4.389838,4.136264,1.764259,-1.310112,-1.460094
4,1.0,5.0,5.699543,-1.828192,-5.436470,-1.820016,-2.514823
5,1.0,6.0,4.233797,3.881289,-1.346438,-0.594609,0.525966
6,1.0,7.0,2.964878,-1.591166,-2.426420,1.396008,-0.279348
7,1.0,8.0,-1.701350,-2.296212,0.940899,0.703827,-0.041181
8,1.0,9.0,0.619941,-0.645922,1.283698,-0.192693,-1.118428
9,1.0,10.0,2.663591,1.607673,1.109612,-0.491242,-0.227089


<h3> Instruments data frame </h3>

In [83]:
pd.DataFrame(np.array(psc_data[1][0]['inst_df']), columns = psc_data[0][1]['Dins_nms'])

,t,W1,W2,W3,W4,W5,W6,W7,W8,W9,W10
0,1.0,-0.497363,-0.773851,-1.198938,-0.579781,-1.323837,-0.817919,-1.140947,-0.983452,-1.484137,-1.363378
1,2.0,-0.755884,-1.312031,-2.135015,0.951309,1.081004,0.585730,-1.412066,-0.069661,0.688647,1.923460
2,3.0,0.138462,-1.740714,-0.764069,-2.653145,-0.498588,-0.488928,1.398567,1.160595,1.651778,0.796100
3,4.0,0.626904,1.640215,0.060663,-0.280395,0.228471,-0.544038,-1.601036,1.670753,0.792948,-0.328958
4,5.0,-1.356401,-1.599046,-0.559568,-0.803863,0.219712,-0.185293,0.486985,1.108595,1.338126,1.996169
5,6.0,-0.470259,1.262776,-0.301247,-0.328295,-1.627829,-0.632116,-1.271702,-0.061856,-0.778098,-1.423087
6,7.0,0.014171,0.578652,-0.234302,-0.110842,-1.920479,-0.068194,-0.303733,-0.607304,-0.866849,-0.119548
7,8.0,0.033646,-0.996905,-0.027382,1.086518,0.879916,0.463995,0.518848,0.619071,1.682154,0.742068
8,9.0,-0.830632,0.211294,-0.432866,-0.186216,1.017942,1.348544,1.715991,0.978932,-0.764979,-1.113031
9,10.0,-0.465770,0.028214,-0.305423,-0.146574,-0.042972,-0.578392,-0.569208,0.870982,0.166180,0.666163


<h3> Error term data frame </h3>

In [84]:
pd.DataFrame(np.array(psc_data[1][0]['err_df']), columns = psc_data[0][1]['Derr_nms'])

,t,"V1,1","V1,2",e1,"V2,1","V2,2",e2,"V3,1","V3,2",e3,"V4,1","V4,2",e4
0,1.0,-0.007958,-0.234905,0.124638,-0.125850,-0.508685,-0.858495,2.779993,1.512980,-0.096124,0.749408,0.142043,-0.312424
1,2.0,-0.397665,-0.356935,0.331863,-1.829045,-1.158791,0.941441,-0.810977,-1.003903,-0.261438,-0.263864,-1.257571,-1.192111
2,3.0,-0.028513,-0.228178,0.533936,1.353842,0.420532,-0.627427,0.734368,0.488122,-0.043066,1.203501,1.440100,1.374084
3,4.0,1.143086,0.934362,0.867851,0.465869,0.346925,0.267468,-0.975966,-1.020284,-1.458268,0.453370,0.457730,0.230692
4,5.0,0.136348,-0.271324,0.396457,1.039171,2.487005,2.444673,-0.318923,-1.256854,-2.166964,0.504956,0.573398,0.764515
5,6.0,0.233212,-0.549639,-0.873354,-0.613967,-0.275518,0.161046,-1.776231,-0.645365,0.213096,-1.602811,-1.253036,-0.345936
6,7.0,-1.573335,-1.359646,-0.545731,-1.177601,-1.251786,-0.322292,1.216374,1.001941,-0.063379,0.043076,0.022249,-0.349988
7,8.0,-0.266071,-0.196446,-0.209247,-0.361892,-1.391759,-1.066830,-0.967810,-0.803927,-1.045661,1.941410,1.142039,0.391538
8,9.0,0.540394,0.819518,0.623826,1.107965,1.779993,1.543474,0.564366,-0.268980,-0.072229,-2.032084,-1.840734,-1.204346
9,10.0,0.706776,1.277142,1.429682,-1.170837,-0.753361,0.348781,-1.345619,-1.211307,-1.558151,0.767406,0.329106,-0.137320
